#### Triple A to MLB Translation

In [1]:
import pandas as pd
import numpy as np
import os

os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/MLB Season Model/') 

In [7]:
minor = pd.read_csv('./minor_leauge_preds/data/minor_clean.csv')
major = pd.read_csv('./major_leauge_preds/data/cleaned/data_02_24.csv')

In [8]:
minor_aaa = minor[minor['Level'] == 'AAA']
minor_aaa = minor_aaa.sort_values(['Name', 'Season'])
minor_aaa = minor_aaa.groupby(['Name', 'Season']).apply(
    lambda x: pd.Series({
        'PA': x['PA'].sum(),
        'Age': x.iloc[-1]['Age'],
        'BB%': (x['BB%'] * x['PA']).sum() / x['PA'].sum(),
        'K%': (x['K%'] * x['PA']).sum() / x['PA'].sum(),
        'OBP': (x['OBP'] * x['PA']).sum() / x['PA'].sum(),
        'SLG': (x['SLG'] * x['PA']).sum() / x['PA'].sum(),
        'ISO': (x['ISO'] * x['PA']).sum() / x['PA'].sum(),
        'BABIP': (x['BABIP'] * x['PA']).sum() / x['PA'].sum(),
        'wRC': (x['wRC'] * x['PA']).sum() / x['PA'].sum(),
        'wRAA': (x['wRAA'] * x['PA']).sum() / x['PA'].sum(),
        'wOBA': (x['wOBA'] * x['PA']).sum() / x['PA'].sum(),
        'wRC+': (x['wRC+'] * x['PA']).sum() / x['PA'].sum(),
    })
).reset_index()

C:\Users\dalto\AppData\Local\Temp\ipykernel_26492\1845016317.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  minor_aaa = minor_aaa.groupby(['Name', 'Season']).apply(


In [ ]:
minor_aaa['Season_diff'] = minor_aaa.groupby('Name')['Season'].diff(1)
minor_aaa['Season_diff'] =  minor_aaa['Season_diff'].fillna(0)
minor_aaa = minor_aaa[minor_aaa['Season_diff'] <= 1]
minor_aaa['Season_diff'] = minor_aaa.groupby('Name')['Season'].diff(1)
minor_aaa['Season_diff'] =  minor_aaa['Season_diff'].fillna(0)
minor_aaa = minor_aaa[minor_aaa['Season_diff'] <= 1]
minor_aaa = minor_aaa.drop(['Season_diff'])


In [10]:
from tools.zscore_pa import normalize
minor_aaa = normalize(minor_aaa)

In [11]:
from tools.three_yr_avg import three_year_avg
minor_aaa = three_year_avg(minor_aaa)

MLB Data Frame

In [ ]:
major = major[major['PA'] > 100]
major = major.sort_values(by=['Name'])
major = three_year_avg(major)
major = major.drop_duplicates(subset='Name', keep='first')
major = major.drop(columns=['Unnamed: 0'])
major['Team'] = 'MLB'
major = major.rename(columns={'Team':'Level'})
major = major[['Name', 'Level', 'wRC+']]

Merge

In [35]:
outcome = minor_aaa.merge(major[['Name','wRC+']], on=['Name'], how='outer')